## About

In this notebook we prepare a simple solution for the [kaggle challenge on higgs.](https://inclass.kaggle.com/c/mlhep-2016-higgs-detection)

In [3]:
%matplotlib inline

In [35]:
import matplotlib.pyplot as plt

import pandas
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

### Download data

In [5]:
!cd datasets; wget -O public_train_10000.root -nc --no-check-certificate https://2016.mlhep.yandex.net/data/higgs/public_train_10000.root

wget: /root/miniconda/envs/rep_py2/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /root/miniconda/envs/rep_py2/lib/libssl.so.1.0.0: no version information available (required by wget)
File `public_train_10000.root' already there; not retrieving.


In [6]:
# you can download training sample with 100000 available events
# uncomment the below row
# !cd datasets; wget -O public_train_100000.root -nc --no-check-certificate https://2016.mlhep.yandex.net/data/higgs/public_train_100000.root

In [7]:
!cd datasets; wget -O public_test.root -nc --no-check-certificate https://2016.mlhep.yandex.net/data/higgs/public_test.root

wget: /root/miniconda/envs/rep_py2/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /root/miniconda/envs/rep_py2/lib/libssl.so.1.0.0: no version information available (required by wget)
File `public_test.root' already there; not retrieving.


### Read the smallest part of training file and test file

In [8]:
import root_numpy
data = pandas.DataFrame(root_numpy.root2array('datasets/public_train_10000.root'))
test = pandas.DataFrame(root_numpy.root2array('datasets/public_test.root'))

In [10]:
pandas.options.display.max_columns = 50
data.head()

,event_id,target,lepton_pt,lepton_eta,lepton_phi,mem_pt,mem_phi,jet1_pt,jet1_eta,jet1_phi,jet1_btag,jet2_pt,jet2_eta,jet2_phi,jet2_btag,jet3_pt,jet3_eta,jet3_phi,jet3_btag,jet4_pt,jet4_eta,jet4_phi,jet4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1000001,1,34.750568,0.787025,1.898891,20.862434,-2.622998,44.814148,-0.168171,2.631595,2.000023,57.689522,-0.161628,-0.682047,0.000000,71.954201,1.154233,-2.858513,2.000016,79.948036,0.877472,-0.256736,0.000000,81.724449,189.583145,80.118317,170.086075,91.128204,298.468781,374.685760
1,1000002,1,54.250927,-1.057915,2.310697,51.167873,2.545749,31.039040,-2.220276,-0.737298,0.000000,52.221519,-1.094289,-0.252777,2.000023,42.725616,1.474829,2.906143,0.000000,71.681404,-1.139118,-1.300325,2.000016,65.837746,201.096756,83.321556,208.039688,67.118484,287.363983,527.247559
2,1000003,1,47.746025,-0.783184,2.660325,68.165527,-1.700790,36.945312,-1.151738,-0.626912,0.000000,118.880913,-0.211263,1.326902,0.000000,40.954044,-1.149437,2.123149,1.000011,113.246666,-0.798898,-1.605550,2.000016,69.316925,156.334732,95.307602,149.089005,130.389206,237.879318,336.058838
3,1000004,0,45.950066,1.613817,0.964722,39.302082,-0.075989,84.307426,0.465748,2.287783,2.000016,46.788040,0.441073,-1.303352,0.000000,15.260404,2.409047,-0.850500,0.000000,30.741282,-0.586711,-2.256246,2.000023,71.032066,182.341537,81.941925,164.411148,93.709511,237.900055,392.807831
4,1000005,0,44.409187,-0.228907,-1.837974,49.886654,0.156533,32.852112,2.561646,2.641460,0.000000,46.283184,-1.488267,-2.524357,2.000000,29.664640,-0.031725,-1.192486,0.000000,138.741928,0.293522,1.391425,0.000000,122.030174,288.594086,84.386459,150.299744,69.818291,435.990356,533.977905


In [11]:
test.head()

,event_id,lepton_pt,lepton_eta,lepton_phi,mem_pt,mem_phi,jet1_pt,jet1_eta,jet1_phi,jet1_btag,jet2_pt,jet2_eta,jet2_phi,jet2_btag,jet3_pt,jet3_eta,jet3_phi,jet3_btag,jet4_pt,jet4_eta,jet4_phi,jet4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1,58.814354,-1.223199,1.290717,26.435696,2.608772,68.413330,0.639561,-1.226549,1.000008,28.192644,1.319475,2.704676,0.000000,90.252411,-0.443325,-0.734594,1.999937,35.200653,-1.195189,1.767687,0.000000,72.190338,170.810608,78.644196,224.813538,95.737434,372.835388,469.654236
1,2,21.390781,-0.468277,-1.371404,57.185963,1.214130,118.127953,-0.113784,-2.182008,0.000000,107.238831,-1.082186,0.232456,2.000016,68.670891,0.110876,2.863325,0.000000,40.473949,0.965376,2.698023,1.000000,62.736309,215.426300,73.971413,211.729141,195.910385,310.480103,431.597137
2,3,58.570217,1.443092,0.105191,54.450954,-2.354347,26.443583,-1.911658,1.337635,0.000000,154.289459,-1.142575,-2.166013,0.999969,139.899277,0.307173,1.089414,2.000016,24.017288,-2.212247,-2.609508,0.000000,48.172768,163.506821,106.111038,260.643646,351.328644,638.053040,790.960815
3,4,70.469345,0.166176,-1.962429,38.138966,2.564890,42.678413,-1.256608,-0.276156,2.000000,42.376274,0.249604,-0.689569,0.000000,61.597656,-0.904709,1.914711,0.000000,37.422001,-0.007441,3.048725,2.000023,99.099815,158.532227,79.701500,169.550491,96.056900,305.073212,402.656067
4,5,113.456779,0.232503,2.941150,64.401146,1.125812,164.289139,-0.735258,-0.590741,1.999937,48.857182,-0.814078,3.020974,0.000000,32.992023,-0.362528,-2.159055,0.000000,37.717247,-0.943712,1.599526,2.000023,50.816051,200.099945,130.013855,187.637009,142.145920,433.032440,507.496399


### Define training features

Exclude `event_id`, `target` from the features set

In [18]:
features = list(set(data.columns) - {'event_id', 'target'})
features_data = data.drop("target", axis=1).astype(numpy.float64)
features

['jet3_pt',
 'jet3_eta',
 'm_jjj',
 'mem_phi',
 'jet1_pt',
 'jet4_phi',
 'jet1_phi',
 'jet2_eta',
 'jet3_btag',
 'm_jlv',
 'm_wbb',
 'jet4_pt',
 'jet4_btag',
 'jet2_pt',
 'jet1_btag',
 'm_jj',
 'm_wwbb',
 'jet2_phi',
 'lepton_phi',
 'm_bb',
 'm_lv',
 'jet4_eta',
 'jet2_btag',
 'lepton_pt',
 'mem_pt',
 'lepton_eta',
 'jet3_phi',
 'jet1_eta']

## Btag jets selection

In [76]:
def second_largest(numbers):
    count = 0
    m1 = m2 = float('-inf')
    for x in numbers:
        count += 1
        if x > m2:
            if x >= m1:
                m1, m2 = x, m1            
            else:
                m2 = x
    return m2 if count >= 2 else None

def delta_R(eta1,eta2,phi1,phi2):
    return np.sqrt((eta1-eta2)**2+(phi1-phi2)**2)

btag_features = list(filter(lambda feature: feature.endswith("_btag"), features_data.columns))
pt_features   = list(filter(lambda feature: feature.endswith("_pt"), features_data.columns))
#print(btag_features)
data_btags = data[btag_features]
np_btags = np.array(btag_features)
#print data_btags[0:10]
#print data[0:10]
del_pts_bb = []
del_phis_bb = []
del_Rs_bb = []
for index, row in data_btags.iterrows():
    max1_index = list(row).index(row.max())
    max2_index = list(row).index(second_largest(row))
    jet_label_1 = 'jet{}_'.format(max1_index+1)
    jet_label_2 = 'jet{}_'.format(max2_index+1)
    jet_max1_pt = data.iloc[index]['{}pt'.format(jet_label_1)]
    jet_max2_pt = data.iloc[index]['{}pt'.format(jet_label_2)]
    jet_max1_eta = data.iloc[index]['{}eta'.format(jet_label_1)]
    jet_max2_eta = data.iloc[index]['{}eta'.format(jet_label_2)]
    jet_max1_phi = data.iloc[index]['{}phi'.format(jet_label_1)]
    jet_max2_phi = data.iloc[index]['{}phi'.format(jet_label_2)]
    del_pt_bb  = np.abs(jet_max1_pt - jet_max2_pt)
    del_phi_bb = np.abs(jet_max1_phi - jet_max2_phi)
    del_R_bb   = delta_R(jet_max1_eta,jet_max2_eta,jet_max1_phi,jet_max2_phi)
    del_pts_bb.append(del_pt_bb)
    del_phis_bb.append(del_phi_bb)
    del_Rs_bb.append(del_R_bb)
    if index < 11:
        print   jet_label_1, jet_label_2, jet_max1_pt, jet_max2_pt, jet_max1_phi, jet_max2_phi, del_pt_bb, del_R_bb, del_phi_bb

#print np.array(del_pts_bb).shape
data['del_pt_bb']=np.array(del_pts_bb)
data['del_phi_bb']=np.array(del_phis_bb)
data['del_R_bb']=np.array(del_Rs_bb)

#data[0:10]

   jet1_btag  jet2_btag  jet3_btag  jet4_btag
0   2.000023   0.000000   2.000016   0.000000
1   0.000000   2.000023   0.000000   2.000016
2   0.000000   0.000000   1.000011   2.000016
3   2.000016   0.000000   0.000000   2.000023
4   0.000000   2.000000   0.000000   0.000000
5   0.000000   1.999937   0.000000   0.000000
6   0.000000   1.000008   0.000000   1.999937
7   2.000023   2.000016   0.000000   0.000000
8   0.000000   2.000000   1.000008   0.000000
9   0.000000   0.000000   1.999937   1.000000
   event_id  target  lepton_pt  lepton_eta  lepton_phi      mem_pt   mem_phi  \
0   1000001       1  34.750568    0.787025    1.898891   20.862434 -2.622998   
1   1000002       1  54.250927   -1.057915    2.310697   51.167873  2.545749   
2   1000003       1  47.746025   -0.783184    2.660325   68.165527 -1.700790   
3   1000004       0  45.950066    1.613817    0.964722   39.302082 -0.075989   
4   1000005       0  44.409187   -0.228907   -1.837974   49.886654  0.156533   
5   1000006   

,event_id,target,lepton_pt,lepton_eta,lepton_phi,mem_pt,mem_phi,jet1_pt,jet1_eta,jet1_phi,jet1_btag,jet2_pt,jet2_eta,jet2_phi,jet2_btag,jet3_pt,jet3_eta,jet3_phi,jet3_btag,jet4_pt,jet4_eta,jet4_phi,jet4_btag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb,del_pt_bb,del_phi_bb,del_R_bb
0,1000001,1,34.750568,0.787025,1.898891,20.862434,-2.622998,44.814148,-0.168171,2.631595,2.000023,57.689522,-0.161628,-0.682047,0.000000,71.954201,1.154233,-2.858513,2.000016,79.948036,0.877472,-0.256736,0.000000,81.724449,189.583145,80.118317,170.086075,91.128204,298.468781,374.685760,27.140053,5.490108,5.647126
1,1000002,1,54.250927,-1.057915,2.310697,51.167873,2.545749,31.039040,-2.220276,-0.737298,0.000000,52.221519,-1.094289,-0.252777,2.000023,42.725616,1.474829,2.906143,0.000000,71.681404,-1.139118,-1.300325,2.000016,65.837746,201.096756,83.321556,208.039688,67.118484,287.363983,527.247559,19.459885,1.047548,1.048506
2,1000003,1,47.746025,-0.783184,2.660325,68.165527,-1.700790,36.945312,-1.151738,-0.626912,0.000000,118.880913,-0.211263,1.326902,0.000000,40.954044,-1.149437,2.123149,1.000011,113.246666,-0.798898,-1.605550,2.000016,69.316925,156.334732,95.307602,149.089005,130.389206,237.879318,336.058838,72.292622,3.728699,3.745140
3,1000004,0,45.950066,1.613817,0.964722,39.302082,-0.075989,84.307426,0.465748,2.287783,2.000016,46.788040,0.441073,-1.303352,0.000000,15.260404,2.409047,-0.850500,0.000000,30.741282,-0.586711,-2.256246,2.000023,71.032066,182.341537,81.941925,164.411148,93.709511,237.900055,392.807831,53.566145,4.544029,4.664319
4,1000005,0,44.409187,-0.228907,-1.837974,49.886654,0.156533,32.852112,2.561646,2.641460,0.000000,46.283184,-1.488267,-2.524357,2.000000,29.664640,-0.031725,-1.192486,0.000000,138.741928,0.293522,1.391425,0.000000,122.030174,288.594086,84.386459,150.299744,69.818291,435.990356,533.977905,13.431072,5.165817,6.564104
5,1000006,1,47.960735,0.113996,1.060799,106.161682,2.015695,60.525757,-0.102381,-2.320091,0.000000,123.111198,0.490156,-1.061288,1.999937,33.649323,0.280342,2.135304,0.000000,42.861736,1.540545,-0.419176,0.000000,77.443741,184.853806,79.668442,140.052399,42.061230,319.923248,412.156982,62.585442,1.258803,1.391290
6,1000007,1,46.442108,0.985508,-1.626641,26.800890,-2.045213,53.416817,0.993135,0.138351,0.000000,55.109737,1.250576,3.003202,1.000008,15.932467,1.188875,-0.360576,0.000000,61.113026,-0.641036,1.519797,1.999937,16.887491,153.891785,75.502884,154.074692,145.112534,285.292542,367.908844,6.003288,1.483405,2.403890
7,1000008,0,40.197048,0.019778,-3.087664,86.059540,1.251372,90.823509,-0.155913,-2.222942,2.000023,92.754059,-0.372654,-0.643558,2.000016,22.154982,1.290531,-0.019670,0.000000,54.376598,-0.481523,1.795838,0.000000,81.358971,175.508392,97.613731,199.411179,126.514870,310.691071,405.147736,1.930550,1.579384,1.594186
8,1000009,0,31.490555,-1.100834,-0.548881,46.351318,0.409934,59.719730,1.354065,2.189174,0.000000,53.383526,-0.126176,-2.366048,2.000000,120.977127,-0.807105,-1.359135,1.000008,77.764938,-0.015128,1.995870,0.000000,74.385422,163.475708,79.864510,186.273666,96.346321,347.697540,455.717010,67.593601,1.006913,1.215541
9,1000010,1,41.040684,0.121702,-1.623338,57.911297,2.293132,56.781654,-0.769391,-1.460335,0.000000,43.869629,-1.114340,0.380104,0.000000,59.761009,0.785001,0.752367,1.999937,64.861122,1.042323,-3.137323,1.000000,78.362534,201.900909,92.482224,150.610504,118.025726,342.895172,401.707764,5.100113,3.889691,3.898193


### Prepare high-level features for training

In [ ]:
high_level_features = ['m_jj', 'm_jjj', 'm_jlv', 'm_wwbb', 'm_bb', 'm_wbb', 'm_lv']

### Plot histograms for each high-level feature

In [ ]:
hist_params = {'normed': True, 'bins': 60, 'alpha': 0.4}
# create the figure
plt.figure(figsize=(16, 25))
for n, feature in enumerate(high_level_features):
    # add sub plot on our figure
    plt.subplot(len(features) // 5 + 1, 3, n+1)
    # define range for histograms by cutting 1% of data from both ends
    min_value, max_value = numpy.percentile(data[feature], [1, 99])
    plt.hist(data.ix[data.target.values == 0, feature].values, range=(min_value, max_value), 
             label='class 0', **hist_params)
    plt.hist(data.ix[data.target.values == 1, feature].values, range=(min_value, max_value), 
             label='class 1', **hist_params)
    plt.legend(loc='best')
    plt.title(feature)

### Divide training data into 2 parts 
`train_test_split` function is used to divide into 2 parts to preserve quality overestimating.

In [ ]:
training_data, validation_data = train_test_split(data, random_state=11, train_size=0.66)

### Simple knn from `sklearn` training

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(training_data[high_level_features], training_data.target)

In [ ]:
# predict validation sample (probability for each event)
proba = knn.predict_proba(validation_data[high_level_features])

In [ ]:
proba

### Compute quality (ROC AUC) on the validation set (to prevent overestimating quality)

In [ ]:
# take probability to be 1 class to compute ROC AUC
roc_auc_score(validation_data.target, proba[:, 1])

## Prepare submission to kaggle

In [ ]:
# predict test sample
kaggle_proba = knn.predict_proba(test[high_level_features])[:, 1]
kaggle_ids = test.event_id

In [ ]:
from IPython.display import FileLink
def create_solution(ids, proba, filename='baseline.csv'):
    """saves predictions to file and provides a link for downloading """
    pandas.DataFrame({'event_id': ids, 'prediction': proba}).to_csv('datasets/{}'.format(filename), index=False)
    return FileLink('datasets/{}'.format(filename))
    
create_solution(kaggle_ids, kaggle_proba)